In [1]:
import random
import numpy as np

from deap import base
from deap import creator
from deap import tools

In [2]:
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

In [3]:
toolbox = base.Toolbox()

toolbox.register("init_gene", random.random)
toolbox.register("init_individual", tools.initRepeat, 
                 creator.Individual, toolbox.init_gene, 9)

toolbox.register("init_population", tools.initRepeat, 
                 list, toolbox.init_individual)

In [4]:
from pyscf.gto import Mole
from pyscf.scf import RHF

def build_molecule_from_genome(genome):
    atoms = [
        ("O", genome[0], genome[1], genome[2]),
        ("H", genome[3], genome[4], genome[5]),
        ("H", genome[6], genome[7], genome[8])
    ]
    
    mol = Mole()
    mol.atom = atoms
    mol.basis = "sto-3g"
    mol.build()
    return mol
    
def evaluateFitness(individual):
    
    mol = build_molecule_from_genome(individual)
    
    mf = RHF(mol)
    mf.verboseose = 1
    E = mf.scf()
    
    # this shit has to be a tuple!!
    return E,

toolbox.register("evaluate", evaluateFitness)

/home/jo/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [5]:
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05) # flip gene with 0.05 % probability
toolbox.register("select", tools.selTournament, tournsize=3)

# Do the Optimazation 

In [6]:
PROBABILITY_CROSSING = 0.5
PROBABILITY_MUTATION = 0.5

MAX_ITERATIONS = 1000

CONVERGENCE_THRESHOLD = 1e-10

E_old = 1e10

In [7]:
SIZE_POPULATION = 30

population = toolbox.init_population(n=SIZE_POPULATION)

In [8]:
fitnesses = list(map(toolbox.evaluate, population))

for ind, fit in zip(population, fitnesses):
    ind.fitness.values = fit

converged SCF energy = -71.6455719977427
converged SCF energy = -74.1075077154401


<class 'pyscf.scf.hf.RHF'> does not have attributes  verboseose


converged SCF energy = -73.7295558970304
converged SCF energy = -72.3708371487364
converged SCF energy = -74.0340098792531
converged SCF energy = -71.7180719641193
converged SCF energy = -70.2043587472147
converged SCF energy = -70.6634744936453
converged SCF energy = -73.4406380725524
converged SCF energy = -71.6716310407989
converged SCF energy = -72.4427715125503
converged SCF energy = -71.5619753397549
converged SCF energy = -71.1006357013045
converged SCF energy = -73.1918384355213
converged SCF energy = -74.1838257509756
converged SCF energy = -74.8831545916949
converged SCF energy = -73.812472905598
converged SCF energy = -72.4111677546145
converged SCF energy = -58.8860237753499
converged SCF energy = -74.5288245757116
converged SCF energy = -72.4137117039943
converged SCF energy = -73.4255780579001
converged SCF energy = -59.7724177999014
converged SCF energy = -74.5996997835119
converged SCF energy = -74.6269008287131
converged SCF energy = -72.5373600498219
converged SCF ene

In [9]:
# create a list of fitness values
fitness_values = [ind.fitness.values[0] for ind in population]

# Begin the evolution
for i in range(MAX_ITERATIONS):

    # Select next generation
    offspring = toolbox.select(population, len(population))
    offspring = list(map(toolbox.clone, offspring))

    # do cross over
    for child1, child2 in zip(offspring[::2], offspring[1::2]):

        if random.random() < PROBABILITY_CROSSING:
            toolbox.mate(child1, child2)

            del child1.fitness.values
            del child2.fitness.values
            
    # do mutation
    for mutant in offspring:
        if random.random() < PROBABILITY_MUTATION:
            toolbox.mutate(mutant)
            del mutant.fitness.values
    
    # recalculate fitness values of mates and mutants
    invalid_individuals = [ind for ind in offspring if not ind.fitness.valid]
    fitnesses = map(toolbox.evaluate, invalid_individuals)
    for ind, fit in zip(invalid_individuals, fitnesses):
        ind.fitness.values = fit
    
    population[:] = offspring
    
    # update list of fitness value
    fitness_values = [ind.fitness.values[0] for ind in population]
    
    #E = sum(fitness_values) / len(fitness_values)
    E = min(fitness_values)
    print(E_old, E)
    
    if np.abs(E - E_old) < CONVERGENCE_THRESHOLD:
        break
    else:
        E_old = E
        
    
    

converged SCF energy = -73.7295558970304
converged SCF energy = -74.8054843094545
converged SCF energy = -74.5288245757116
converged SCF energy = -74.1854916140017
converged SCF energy = -74.6968703272065
converged SCF energy = -70.7987484684045
converged SCF energy = -74.0802315422683
converged SCF energy = -73.131612027342
converged SCF energy = -74.1838257509756
converged SCF energy = -74.1838257509756
converged SCF energy = -74.8117424130229
converged SCF energy = -73.1576455860665
converged SCF energy = -66.1322678155239
converged SCF energy = -72.6632911789324
converged SCF energy = -74.7105030070539
converged SCF energy = -74.5744439637276
converged SCF energy = -73.3990450204173
converged SCF energy = -73.4406380725524
converged SCF energy = -72.4111677546145
converged SCF energy = -74.8831545916949
converged SCF energy = -74.5996997835119
10000000000.0 -74.8831545916949
converged SCF energy = -74.731116378314
converged SCF energy = -66.6898239456287
converged SCF energy = -73.

In [10]:
best_ind = tools.selBest(population, 1)[0]
print(best_ind.fitness.values)

(-74.91427817847651,)


# Checking the Result 

In [11]:
import matplotlib.pylab as plt

x = [best_ind[0], best_ind[3], best_ind[6]]
y = [best_ind[1], best_ind[4], best_ind[7]]
z = [best_ind[2], best_ind[5], best_ind[8]]

lower, upper = -0.5, 1.5

plt.subplot(2, 2, 1)
plt.scatter(x[0], y[0], label="C")
plt.scatter(x[1:], y[1:], label="H")
plt.xlabel("x")
plt.ylabel("y")
plt.xlim(lower, upper)
plt.ylim(lower, upper)

plt.subplot(2, 2, 2)
plt.scatter(x[0], z[0], label="C")
plt.scatter(x[1:], z[1:], label="H")
plt.xlabel("x")
plt.ylabel("z")
plt.xlim(lower, upper)
plt.ylim(lower, upper)

plt.subplot(2, 2, 3)
plt.scatter(y[0], z[0], label="C")
plt.scatter(y[1:], z[1:], label="H")
plt.xlabel("y")
plt.ylabel("z")
plt.xlim(lower, upper)
plt.ylim(lower, upper)

plt.show()

In [12]:
C  = np.array([x[0], y[0], z[0]])
H1 = np.array([x[1], y[1], z[1]])
H2 = np.array([x[2], y[2], z[2]])


CH1 = C - H1
CH2 = C - H2
distance_C_H1 = np.linalg.norm(CH1)
distance_C_H2 = np.linalg.norm(CH2)

angle = np.arccos(np.dot(CH1, CH2) / (distance_C_H1 * distance_C_H2))

print("Distance C-H1: {0},\nDistance C-H2: {1},\nAngle Distance C-H1: {2}".format(
    distance_C_H1,
    distance_C_H2,
    angle / (2 * np.pi) * 360
))

Distance C-H1: 0.9610121447864596,
Distance C-H2: 1.2172840817290067,
Angle Distance C-H1: 82.97935170899447
